In [2]:
def xpath_to_css(xpath):
    """
    XPathをCSSセレクタに変換する関数
    
    Args:
        xpath (str): 変換したいXPath式
        
    Returns:
        str: CSSセレクタ
        
    対応しているXPath構文:
    - 要素選択 (div, p, etc.)
    - クラス属性 (@class)
    - ID属性 (@id)
    - 属性選択 ([@attr='value'])
    - インデックス選択 ([n])
    - 子要素 (/)
    - 子孫要素 (//)
    """
    import re
    
    # 子孫セレクタ (//) をスペースに変換
    css = xpath.replace('//', ' ')
    
    # 子セレクタ (/) を > に変換
    css = css.replace('/', ' > ')
    
    # 属性セレクタの変換
    css = re.sub(r'\[@([a-zA-Z_-]+)=[\'"](.+?)[\'"]\]', r'[\1="\2"]', css)
    
    # @classを.に変換
    css = re.sub(r'\[@class=[\'"](.+?)[\'"]\]', r'.\1', css)
    css = css.replace('.[', '[')  # クラスが複数ある場合の調整
    
    # @idを#に変換
    css = re.sub(r'\[@id=[\'"](.+?)[\'"]\]', r'#\1', css)
    
    # インデックスセレクタの変換 [n] → :nth-of-type(n)
    css = re.sub(r'\[(\d+)\]', r':nth-of-type(\1)', css)
    
    # 余分な空白を削除
    css = ' '.join(css.split())
    
    # 最初の > を削除 (html > body などが > html body になるのを防ぐ)
    if css.startswith('> '):
        css = css[2:]
    
    return css.strip()


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

url = "https://univ-online.com/rank3/"  # 対象URL

# /html/body/div/div/div/div/div/main/div/section[33]
# /html/body/div/div/div/div/div/main/div/section[1]/h3
for i in range(1):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    type_css_selector = xpath_to_css(f'/html/body/div/div/div/div/div/main/div/section[{i+1}]/h3')
    type_name = soup.select_one(type_css_selector)
    print(type_name.text)
    css_selector = xpath_to_css(f'/html/body/div/div/div/div/div/main/div/section[{i+1}]/ul')
    elements = list(soup.select(css_selector))
    if not elements:
        continue
    for j in range(str(elements[0]).count('<li class="univ-list__item univ-list__item_col_4">')):
        # /html/body/div/div/div/div/div/main/div/section[]/ul/li[]/a
        css_selector_child = xpath_to_css(f'/html/body/div/div/div/div/div/main/div/section[{i+1}]/ul/li[{j+1}]/a')
        link_element = soup.select_one(css_selector_child)
        print(link_element.get_text(strip=True))
        name = link_element.get_text(strip=True)

        if link_element and 'href' in link_element.attrs:
            href = link_element['href']
            # print("取得したリンク ab:", href)
            # 相対URLの場合、ベースURLと結合
            if not href.startswith(('http://', 'https://')):
                href = requests.compat.urljoin(url, href)
            # else:
            #     print("リンクが見つかりませんでした")
            # print("取得したリンク re:", href)

        # リンク先に移動（新しいリクエストを送信）
        new_response = requests.get(href)
        new_soup = BeautifulSoup(new_response.text, 'html.parser')                

        # 必要に応じて new_response.text を解析
        xpath = '/html/body/div/div/div/div/div/main/div/div[2]/div/table/tbody'
                #/html/body/div/div/div/div/div/main/div/div[]/div/table/tbody
        css_selector_new = xpath_to_css(xpath)

        # XPathをCSSセレクタに変換して使用
        element = new_soup.select(css_selector_new)
        # print(element)
        new_soup = BeautifulSoup(str(element), 'html.parser')
        rows = new_soup.find_all('tr')
        # print(rows)

        data = []
        for row in rows:
            cols = row.find_all('td')
            data.append([col.get_text(strip=True) for col in cols])

        df = pd.DataFrame(data, columns=['Column1', 'Column2', 'Column3', 'Column4', 'Column5'])
        folder_path = f"data/{type_name.text}"

        # フォルダが存在しない場合に作成
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        # print(df)
        df.to_excel(f'data/{type_name.text}/{name}.xlsx', index=False)



NameError: name 'xpath_to_css' is not defined